# OpenAI Code Generation

# Overview  
OpenAI Code Generation allows you to generate code snippets based on given instructions or prompts. \
This feature enables you to create code assistants, automated coding tools, and other developer-centric AI applications by providing a series of instructions as input and receiving a model-generated code snippet as output.\
\
For more detailed examples, refer to the official Training on [Generate code with Azure OpenAI Service](https://learn.microsoft.com/en-us/training/modules/generate-code-azure-openai/)


### 1. Import helper libraries and instantiate credentials and model

In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)
CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME') # model = "deployment_name"

## Generating SQL Queries
In this example, we will use OpenAI to generate a SQL query that lists the names of departments which employed more than 10 employees in the last 3 months.

#### Model Parameters:
 _temperature_: \
  Controls the randomness of the output. A value of 0 makes the output more deterministic.

 _max_tokens_:\
  The maximum number of tokens in the generated output.
  
   _top_p_: \
  Controls diversity via nucleus sampling.

_frequency_penalty and presence_penalty_: \
  These parameters are set to 0 to avoid penalizing the generation of similar words.

 _stop_: \
  Specifies stopping sequences to end the generated text appropriately.

In [2]:
prompt = "### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)
response = client.chat.completions.create(
  model = CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": prompt}],
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response.choices[0].message.content)


### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months

 query: 
To list the names of the departments which employed more than 10 employees in the last 3 months, you can use the following SQL query. This query assumes that the "last 3 months" is relative to the current date.

```sql
SELECT d.name
FROM Department d
JOIN Employee e ON d.id = e.department_id
WHERE e.id IN (
    SELECT DISTINCT e.id
    FROM Employee e
    JOIN Salary_Payments sp ON e.id = sp.employee_id
    WHERE sp.date >= CURRENT_DATE - INTERVAL '3 months'
)
GROUP BY d.id, d.name
HAVING COUNT(DISTINCT e.id) > 10


#### Use cases to explore
1. **Automated Query Generation** \
   Quickly generate complex SQL queries based on natural language descriptions.

2. **Code Assistance** \
   Provide developers with code suggestions and examples to speed up development.

3. **Educational Tools for Developers** \
   Help developers learn SQL by providing examples and explanations of different queries.

## Explaining SQL Code
OpenAI can help you understand and explain code snippets by providing natural language descriptions of what the code does.  \
This feature is useful for creating educational tools, code documentation, and automated code review systems.

In [3]:
code = "Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "

response = client.chat.completions.create(
  model=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": code}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response.choices[0].message.content)


Certainly! Let's break down the SQL query step by step to understand what it does:

1. **FROM and JOIN Clause**:
   - `FROM Department d JOIN Employee e ON d.id = e.department_id`: This part of the query is joining two tables: `Department` and `Employee`. It uses the `department_id` from the `Employee` table to match with the `id` in the `Department` table. This allows us to associate each employee with their respective department.

2. **WHERE Clause**:
   - `WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months')`: This subquery selects `employee_id` from the `Salary_Payments` table where the `date` of the payment is within the last 3 months. The `WHERE` clause then filters the employees to only include those who have received a salary payment in the last 3 months.

3. **GROUP BY and HAVING Clause**:
   - `GROUP BY d.name HAVING COUNT(*) > 10`: After filtering, the query groups the results by department name (`d.name`). The `HAVING` clause is u

#### Use cases to explore
1. **Educational Tools** \
Help your developers understand SQL queries even better by providing clear explanations.

2. **Code Documentation** \
Automatically generate documentation for SQL queries to improve code readability and maintainability.


3. **Automated Code Review** \
Enhance code review processes by providing automated explanations and insights into the code.